In [5]:
# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('./Walmart/small_copy.csv')
df

,Column1,Store,Date,IsHoliday,Dept,Weekly_Sales,Temperature,Type,Size
0,341666,36,05-02-10,0,94,70593.01,45.97,3,39910
1,341667,36,05-02-10,0,92,63964.12,45.97,3,39910
2,341672,36,05-02-10,0,93,39005.54,45.97,3,39910
3,341684,36,05-02-10,0,95,40389.37,45.97,3,39910
4,341693,36,12-02-10,1,94,69820.88,46.11,3,39910
...,...,...,...,...,...,...,...,...,...
580,357499,38,31-12-10,1,24,6.37,45.64,1,39690
581,357519,38,31-12-10,1,95,30113.97,45.64,1,39690
582,357522,38,31-12-10,1,94,14620.77,45.64,1,39690
583,357536,38,31-12-10,1,93,13678.28,45.64,1,39690


In [8]:
df.describe()
grouped = (df.groupby(['Store', 'Date']))['Weekly_Sales'].sum().reset_index()
grouped

,Store,Date,Weekly_Sales
0,36,01-10-10,183379.46
1,36,02-04-10,195281.44
2,36,02-07-10,198326.15
3,36,03-09-10,193543.49
4,36,03-12-10,143740.45
...,...,...,...
139,38,28-05-10,109513.78
140,38,29-10-10,109846.13
141,38,30-04-10,112451.31
142,38,30-07-10,109799.69


In [10]:
df.describe()
grouped2 = (df.groupby(['Store', 'Date'])).agg({
    'Weekly_Sales': 'sum',
    'Temperature': 'mean',
    'Type': 'mean',
    }).reset_index()
grouped2

,Store,Date,Weekly_Sales,Temperature,Type
0,36,01-10-10,183379.46,74.66,3.0
1,36,02-04-10,195281.44,63.43,3.0
2,36,02-07-10,198326.15,81.85,3.0
3,36,03-09-10,193543.49,82.29,3.0
4,36,03-12-10,143740.45,54.44,3.0
...,...,...,...,...,...
139,38,28-05-10,109513.78,67.84,1.0
140,38,29-10-10,109846.13,61.09,1.0
141,38,30-04-10,112451.31,67.38,1.0
142,38,30-07-10,109799.69,96.31,1.0


In [7]:
pivot = grouped.pivot(index='Store', columns='Dept', values='Weekly_Sales')
pivot

Dept,24,92,93,94,95
Store,,,,,
36,82.00,2639650.32,1799058.87,3001343.61,2152225.42
37,6.00,2765189.11,975273.66,1583758.68,2416344.17
38,32.87,1990139.07,812810.77,959907.41,1863836.98


In [ ]:
def smart_aggregate(df, group_cols=['Store', 'Date'], sum_col='Weekly_Sales'):
    """
    Automatically handle columns based on their consistency
    """
    # Get all non-grouping columns except the sum column
    other_cols = [col for col in df.columns 
                  if col not in group_cols and col != sum_col]
    
    agg_dict = {sum_col: 'sum'}  # Always sum the sales
    
    for col in other_cols:
        # Check if column has consistent values within groups
        is_consistent = df.groupby(group_cols)[col].nunique().max() == 1
        
        if is_consistent:
            # Take first value (all are same)
            agg_dict[col] = 'first'
        else:
            print(f"Column '{col}' has conflicting values within groups.")
            # Choose appropriate aggregation based on dtype
            if pd.api.types.is_numeric_dtype(df[col]):
                agg_dict[col] = 'mean'  # Average for numeric
            else:
                # For categorical conflicts, take mode or flag
                agg_dict[col] = lambda x: x.mode()[0] if len(x.mode()) == 1 else 'MULTIPLE_VALUES'
    
    return df.groupby(group_cols).agg(agg_dict).reset_index()

# Usage
grouped = smart_aggregate(df, ['Store', 'Date'], 'Weekly_Sales')
print(grouped.head())

   Store      Date  Weekly_Sales    Column1  IsHoliday  Dept  Temperature  \
0     36  01-10-10     183379.46  343073.25          0  93.5        74.66   
1     36  02-04-10     195281.44  341982.75          0  93.5        63.43   
2     36  02-07-10     198326.15  342508.50          0  93.5        81.85   
3     36  03-09-10     193543.49  342903.00          0  93.5        82.29   
4     36  03-12-10     143740.45  343489.60          0  79.6        54.44   

   Type   Size  
0     3  39910  
1     3  39910  
2     3  39910  
3     3  39910  
4     3  39910  
